Seleção de Features

Neste tópico abordaremos a seleção de features, utilizando o método Botura e teste de Chi2 para tal.

Chi2*: Método utilizado para seleção de features categóricas ordinais e nominais. É um metódo que avalia a indepedência das features em relação ao target.

Boruta: Método muito robusto e baseado em modelo, que leva em consideração a interação entre a features, algo nao levado em conta pelo Chi2 ou outros métodos univariados de seleção.

## Depêndencias

In [1]:
#Importando bibliotecas
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cloudpickle
import matplotlib.pyplot as plt
from scipy.stats import norm
from pandas.api.types import is_numeric_dtype
from src.selecao_de_features import boruta_selector,chi_squared
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

## Dados e Constantes

In [21]:
DATA_RAW_PATH = os.path.join('..','data','raw')
DATA_RAW_NAME = 'diabetes_data.csv'
DATA_INTER_PATH = os.path.join('..','data','inter')
DATA_INTER_TEST_NAME = 'test_inter.csv'
DATA_INTER_TRAIN_NAME = 'train_inter.csv'
DATA_INTER_TRAIN_FT_SELECTION = 'train_inter_ft_selection.csv'
META_DATA_PATH = os.path.join(DATA_RAW_PATH,'dicionario_tipos.pkl')

In [3]:
#Dados do dicionário
with open(META_DATA_PATH,'rb') as f:
    dicionario_tipo = cloudpickle.load(f)
df_train = pd.read_csv(os.path.join(DATA_INTER_PATH, DATA_INTER_TRAIN_NAME), dtype=dicionario_tipo)

In [4]:
df_train.head()

,idade,genero,polyuria,polydipsia,perda_de_peso_repentina,fraqueza,polyphagia,candidíase_genital,desfoque_visual,coceira,irritabilidade,cicatrização_retardada,paresia_parcial,rigidez_muscular,alopecia,obesidade,target
0,67,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0
1,55,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1
2,60,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
3,48,1,1,1,0,1,0,0,1,1,0,1,1,0,0,0,1
4,31,1,1,1,1,1,1,0,0,1,1,0,1,1,0,0,1


## Iniciando análises para seleção

In [10]:
#Utilizando apenas as variaveis numericas.
todas_as_variaveis = set(df_train.columns.tolist())
variaveis_categoricas = set(df_train.select_dtypes(include=['object']).columns.tolist())
variaveis_numericas = todas_as_variaveis - variaveis_categoricas
variaveis_categoricas = list(variaveis_categoricas)

In [11]:
variaveis_categoricas

['polyuria',
 'irritabilidade',
 'fraqueza',
 'perda_de_peso_repentina',
 'desfoque_visual',
 'coceira',
 'polydipsia',
 'polyphagia',
 'rigidez_muscular',
 'alopecia',
 'cicatrização_retardada',
 'paresia_parcial',
 'genero',
 'target',
 'obesidade',
 'candidíase_genital']

## Abordagem (Chi2+Boruta)

## Chi2

In [12]:
chi2_df, logs = chi_squared(df_train, y ='target', cols = variaveis_categoricas)

In [13]:
chi2_df

,column,p-value,chi2_value
0,polyuria,2.743055e-47,208.623267
1,irritabilidade,1.394289e-08,32.195282
2,fraqueza,3.977307e-06,21.275753
3,perda_de_peso_repentina,1.692650e-20,86.120894
4,desfoque_visual,2.937352e-07,26.290466
5,coceira,7.858988e-01,0.073788
6,polydipsia,3.586249e-40,176.019213
7,polyphagia,2.445415e-14,58.136541
8,rigidez_muscular,1.347944e-02,6.105073
9,alopecia,7.310311e-08,28.980698


In [14]:
logs

['A coluna target não pode ser avaliada. ']

Com é o próprio target, não calculamos o chi2 para ele.
* Hipótese nula - H0: As variáveis são independentes) Segundo o teste de hipotése realizado, as variáveis cujo p-valor < 0.05 tem relação com o target.

In [15]:
#As variáveis que serão excluídas. 
lista_descart_chi2 = chi2_df[chi2_df['p-value']>=0.05]['column'].tolist()

In [16]:
lista_descart_chi2

['coceira', 'cicatrização_retardada']

## Boruta

In [18]:
#Vamos filtrar as Features removidas no step anterior antes de passarmos para o Boruta
list_to_drop_boruta = boruta_selector(df_train.drop(lista_descart_chi2,axis=1), y='target')

In [20]:
#Lista de dropagem Geral (chi2 + boruta)
lista_drop_geral = set(lista_descart_chi2+list_to_drop_boruta)
lista_drop_geral

{'candidíase_genital',
 'cicatrização_retardada',
 'coceira',
 'desfoque_visual',
 'fraqueza',
 'obesidade',
 'rigidez_muscular'}

In [22]:
# dropando as colunas pela abordagem.
df_train.drop(lista_drop_geral,axis=1).to_csv(os.path.join(DATA_INTER_PATH,DATA_INTER_TRAIN_FT_SELECTION), index=False)

A abordagem para seleção de features que utilizamos são (Chi2 e Boruta), que combinados trazem resultados bastante relevantes. As features descartadas foram:`candidíase`,`cicatrização_retardada`,`coceira`,`desfoque_visual`, `fraqueza`,`obesidade`, `regidez_muscular`. Todas essas features não serão consideradas para nosso modelo.